In [ ]:
import tushare as ts
import pandas as pd
import numpy as np
import datetime
ts.set_token('7e67081f05f2a9ea58ca60da55f6cb5f62fadd112bf0581d35c9def0')
pro=ts.pro_api()

In [ ]:
#获取所有一级分类
classify2 = list(pro.index_classify(level='L2', src='SW')['index_code'])

In [ ]:
#获取所有分类最新的对应股票（包括进入时间）
#tushare该接口每分钟只能访问100次，获取数据分为两段（二级分类有104个）
divide2 = pro.index_member(index_code=classify2[0])
for i in classify2[1:50]:
    df2 = pro.index_member(index_code=i)
    divide2=divide2.append(df2)

In [ ]:
for j in classify2[50:]:
    df2 = pro.index_member(index_code=j)
    divide2=divide2.append(df2)
Stocklist=list(divide2['con_code'])

In [ ]:
#获取所有分类历史的对应股票（包括进入时间和退出时间）
#tushare该接口每分钟只能访问100次，获取数据分为两段（二级分类有104个）
divide2_old = pro.index_member(index_code=classify2[0],is_new='N')
for i in classify2[1:50]:
    df2 = pro.index_member(index_code=i,is_new='N')
    divide2_old=divide2_old.append(df2)


In [ ]:
for j in classify2[50:]:
    df2 = pro.index_member(index_code=j,is_new='N')
    divide2_old=divide2_old.append(df2)

In [ ]:
#最新与历史合并 剔除退出事件早于20090909的数据，将最新的分类数据的退出时间设为99999999
allclass2=divide2.append(divide2_old)
allclass2=allclass2.fillna(99999999)
allclass2['out_date'] = allclass2['out_date'].apply(int)
allclass2['in_date'] = allclass2['in_date'].apply(int)
data2=allclass2[allclass2['out_date']>20090908]
a=pro.trade_cal(start_date='20090909',end_date='20190909',is_open='1')
a['cal_date'] = a['cal_date'].apply(int)
date=list(a['cal_date'])

In [ ]:
Dataneeded=pd.DataFrame(date,columns=['cal_date'])
Dataneeded.set_index(['cal_date'], inplace=True)

In [ ]:
#获取每只股票每天对应数据
#将index_name改为index_code可获取分类对应代码
for i in Stocklist:
    stock=data2.loc[data2['con_code']==i]
    stock=stock.sort_values('in_date',ascending=False)
    cla_num=len(stock)
    #print(cla_num)
    indate=list(stock['in_date'])
    outdate=list(stock['out_date'])
    indexname=list(stock['index_name'])
    if cla_num==0:
        Dataneeded[i]=np.nan
    if cla_num==1:
        Dataneeded[i]=list(stock['index_name'])[0]
    else:
        Dataneeded.loc[indate[0]:,i]=indexname[0]
        if indate[-1]<20090909:
            Dataneeded.loc[20090909:outdate[-1],i]=indexname[-1]
            for j in range(1,cla_num-1):
                try:
                    Dataneeded.loc[indate[j]:outdate[j],i]=indexname[j]
                except:
                    pass
        else:
            for j in range(1,cla_num):
            
                try:
                    Dataneeded.loc[indate[j]:outdate[j],i]=indexname[j]
                except:
                    pass       


In [ ]:
#导入excel
Dataneeded.to_excel('Class1.xlsx')